In [21]:

import requests
import json
import pandas as pd
import numpy as np
'''C4266 Seattle-Tacoma-Bellevue, WA MSA QCEW Area Code
C3890 Portland-Vancouver-Hillsboro, OR-WA MSA QCEW Area Code'''
# headers = {'Content-Type': 'application/json'}

# import urllib.request
# import urllib
# def qcewCreateDataRows(csv):
#     dataRows = []
#     try: dataLines = csv.decode().split('\r\n')
#     except er: dataLines = csv.split('\r\n')
#     for row in dataLines:
#         dataRows.append(row.split(','))
#     return dataRows

# def qcewGetAreaData(year,qtr,area):
#     urlPath = "http://data.bls.gov/cew/data/api/[YEAR]/[QTR]/area/[AREA].csv"
#     urlPath = urlPath.replace("[YEAR]",year)
#     urlPath = urlPath.replace("[QTR]",qtr.lower())
#     urlPath = urlPath.replace("[AREA]",area.upper())
#     httpStream = urllib.request.urlopen(urlPath)
#     csv = httpStream.read()
#     httpStream.close()
#     return qcewCreateDataRows(csv)

def qcewGetAreaUrl(year,qtr,area):
    urlPath = "https://data.bls.gov/cew/data/api/[YEAR]/[QTR]/area/[AREA].csv"
    urlPath = urlPath.replace("[YEAR]",year)
    urlPath = urlPath.replace("[QTR]",qtr.lower())
    urlPath = urlPath.replace("[AREA]",area.upper())
    return urlPath

URLPATH = qcewGetAreaUrl("2025", "1", "C4266")
#httpStream = urllib.request.urlopen(URLPATH)
#httpStreamData = httpStream.read()



In [22]:
# Area code metadata
AREA_CODES = {
    "C4266": {
        "code": "C4266",
        "name": "Seattle-Tacoma-Bellevue",
        "state": "WA",
        "region": "Seattle-Tacoma-Bellevue, WA MSA",
        "type": "MSA"
    },
    "C3890": {
        "code": "C3890",
        "name": "Portland-Vancouver-Hillsboro",
        "state": "OR-WA",
        "region": "Portland-Vancouver-Hillsboro, OR-WA MSA",
        "type": "MSA"
    }
}


In [28]:
# Example: load a QCEW area CSV using the cache utilities
from bls_housing import load_area_df, get_cached_path, fetch_area_csv

area = "C4266"
year = "2025"
qtr = "1"

# Load DataFrame (uses cache if available)
df = load_area_df(area, year, qtr)
print(f"Loaded rows: {len(df)}")

# Show cached path (or None)
cached = get_cached_path(area, year, qtr)
print("Cached file:", cached)

# Quick peek and filter to MSA totals (agglvl_code == 40)
print(df.head())
msa = df[df.get('agglvl_code') == 40]
print(f"MSA rows: {len(msa)}")
print(msa.head())


Loaded rows: 1954
Cached file: data/cache/C4266_2025_1.csv
  area_fips  own_code industry_code  agglvl_code  size_code  year  qtr  \
0     C4266         0            10           40          0  2025    1   
1     C4266         1            10           41          0  2025    1   
2     C4266         2            10           41          0  2025    1   
3     C4266         3            10           41          0  2025    1   
4     C4266         5            10           41          0  2025    1   

  disclosure_code  qtrly_estabs  month1_emplvl  ...  oty_month3_emplvl_chg  \
0             NaN        125292        2065846  ...                   1504   
1             NaN           347          33063  ...                    263   
2             NaN           219          66778  ...                   1672   
3             NaN           606         182829  ...                   2588   
4             NaN        124120        1783176  ...                  -3019   

   oty_month3_emplvl_pct_ch

In [23]:
# print(httpStreamData)
pdf = pd.read_csv(URLPATH)
# print(pdf[0])
# print(pdf)
# httpStreamDataRows = qcewCreateDataRows(httpStreamData)

# for row in httpStreamDataRows:
#     print(row)

In [ ]:
# important values, agglvl_code 40 is MSA Total covered, own_code 0 is total covered, industry_code 10 is all industries, size_code 0 is All establishment sizes.
filtered_df = pdf[pdf['agglvl_code'] == 40]
print(filtered_df)


  area_fips  own_code industry_code  agglvl_code  size_code  year  qtr  \
0     C4266         0            10           40          0  2025    1   

  disclosure_code  qtrly_estabs  month1_emplvl  ...  oty_month3_emplvl_chg  \
0             NaN        125292        2065846  ...                   1504   

   oty_month3_emplvl_pct_chg  oty_total_qtrly_wages_chg  \
0                        0.1                 3351220668   

   oty_total_qtrly_wages_pct_chg  oty_taxable_qtrly_wages_chg  \
0                            5.7                   1099420779   

   oty_taxable_qtrly_wages_pct_chg oty_qtrly_contributions_chg  \
0                              2.8                    36340094   

   oty_qtrly_contributions_pct_chg  oty_avg_wkly_wage_chg  \
0                              8.2                    116   

   oty_avg_wkly_wage_pct_chg  
0                        5.3  

[1 rows x 42 columns]
